In [1]:
import rosbag_pandas
import pandas as pd
import numpy as np

file_1 = '/home/rik/data/2020_08_05_gannertshofen/estimator_development/sensors_2020-08-05-13-13-56_estimator_2020-08-30-17-48-25_w_baseline.bag'
df_1 = rosbag_pandas.bag_to_dataframe(file_1)

In [19]:
file_2 = '/home/rik/data/2020_08_05_gannertshofen/estimator_development/sensors_2020-08-05-13-13-56_estimator_2020-08-31-09-22-44_spp.bag'
df_2 = rosbag_pandas.bag_to_dataframe(file_2)

In [20]:
import pandas as pd
def getPoseTf(df, topic):
    df_pose = pd.to_datetime(df[topic + '/header/stamp/secs'], unit='s') + pd.to_timedelta(df[topic + '/header/stamp/nsecs'], unit='ns') 
    df_pose = pd.concat([df_pose, df[topic + '/pose/position/x']], axis=1)
    df_pose = pd.concat([df_pose, df[topic + '/pose/position/y']], axis=1)
    df_pose = pd.concat([df_pose, df[topic + '/pose/position/z']], axis=1)
    df_pose = pd.concat([df_pose, df[topic + '/pose/orientation/x']], axis=1)
    df_pose = pd.concat([df_pose, df[topic + '/pose/orientation/y']], axis=1)
    df_pose = pd.concat([df_pose, df[topic + '/pose/orientation/z']], axis=1)
    df_pose = pd.concat([df_pose, df[topic + '/pose/orientation/w']], axis=1)
    df_pose.reset_index(inplace=True)
    df_pose.columns = ['t_arrival', 't', 'x', 'y', 'z', 'q_x', 'q_y', 'q_z', 'q_w']
    df_pose.dropna(inplace=True)
    df_pose.reset_index(inplace=True)
    df_pose.drop('t_arrival', axis=1, inplace=True)
    df_pose.drop('index', axis=1, inplace=True)
    
    from scipy.spatial.transform import Rotation as R
    ypr = df_pose.apply(lambda row: R.from_quat([row.q_x, row.q_y, row.q_z, row.q_w]).as_euler('ZYX', degrees=True), axis=1)
    ypr = pd.DataFrame(ypr.values.tolist(), columns=['yaw', 'pitch', 'roll'])
    df_pose = pd.concat([df_pose, ypr], axis=1)
    df_pose.set_index('t', inplace=True)
    return df_pose

In [21]:
import pandas as pd
def getBias(df, topic):
    df_bias = pd.to_datetime(df[topic + '/header/stamp/secs'], unit='s') + pd.to_timedelta(df[topic + '/header/stamp/nsecs'], unit='ns') 
    df_bias = pd.concat([df_bias, df[topic + '/vector/x']], axis=1)
    df_bias = pd.concat([df_bias, df[topic + '/vector/y']], axis=1)
    df_bias = pd.concat([df_bias, df[topic + '/vector/z']], axis=1)
    df_bias.reset_index(inplace=True)
    df_bias.columns = ['t_arrival', 't', 'x', 'y', 'z']
    df_bias.dropna(inplace=True)
    df_bias.reset_index(inplace=True)
    df_bias.drop('t_arrival', axis=1, inplace=True)
    df_bias.drop('index', axis=1, inplace=True)
    return df_bias

In [22]:
def getHeading(df, topic):
    df_heading = pd.to_datetime(df[topic + '/header/stamp/secs'], unit='s') + pd.to_timedelta(df[topic + '/header/stamp/nsecs'], unit='ns')
    df_heading = pd.concat([df_heading, df[topic + '/position/position/x']], axis=1)
    df_heading = pd.concat([df_heading, df[topic + '/position/position/y']], axis=1)
    df_heading.reset_index(inplace=True)
    df_heading.columns = ['t_arrival', 't', 'base_x', 'base_y']
    df_heading.dropna(inplace=True)
    df_heading.set_index('t', inplace=True)
    df_heading.drop('t_arrival', axis=1, inplace=True)
    # Convert NED->ENU
    import numpy as np
    x = df_heading['base_y'].values
    y = df_heading['base_x'].values

    from scipy.spatial.transform import Rotation as R
    r = R.from_rotvec(np.pi/2 * np.array([0, 0, 1]))
    vectors = np.array([x, y, np.zeros(len(x))]).transpose()
    heading_vectors = r.apply(vectors)

    heading = np.arctan2(heading_vectors[:, 1], heading_vectors[:, 0]) * 180.0 / np.pi

    df_heading['rtk heading'] = heading
    
    return df_heading

In [23]:
df_pose_1 = getPoseTf(df_1, '/moa/mav_state_estimator/optimization')
df_pose_2 = getPoseTf(df_2, '/moa/mav_state_estimator/optimization')
df_heading = getHeading(df_1, '/moa/piksi/attitude_receiver_0/ros/baseline_ned')

df_acc_bias_1 = getBias(df_1, '/moa/mav_state_estimator/acc_bias')
df_acc_bias_2 = getBias(df_2, '/moa/mav_state_estimator/acc_bias')

df_gyro_bias_1 = getBias(df_1, '/moa/mav_state_estimator/gyro_bias')
df_gyro_bias_2 = getBias(df_2, '/moa/mav_state_estimator/gyro_bias')

In [14]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib qt

fontsize=12
fig, axs = plt.subplots(nrows=1, sharex=True)

df_pose_1['yaw'].plot(ax=axs)
df_pose_2['yaw'].plot(ax=axs)
df_heading['rtk heading'].plot(style='-', ax=axs)
axs.legend(['heading 1', 'heading 2', 'heading rtk'])
axs.set_xlabel('Timestamp', fontsize=fontsize)
axs.set_ylabel('Angle [deg]', fontsize=fontsize)

Text(0,0.5,'Angle [deg]')

In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib qt

fontsize=12
fig, axs = plt.subplots(nrows=3, sharex=True)

df_pose_1['x'].plot(ax=axs[0])
df_pose_2['x'].plot(ax=axs[0])
axs[0].set_xlabel('Timestamp', fontsize=fontsize)
axs[0].set_ylabel('Position [m]', fontsize=fontsize)
axs[0].legend(['x 1', 'x 2'])

df_pose_1['y'].plot(ax=axs[1])
df_pose_2['y'].plot(ax=axs[1])
axs[1].set_xlabel('Timestamp', fontsize=fontsize)
axs[1].set_ylabel('Position [m]', fontsize=fontsize)
axs[1].legend(['y 1', 'y 2'])

df_pose_1['z'].plot(ax=axs[2])
df_pose_2['z'].plot(ax=axs[2])
axs[2].set_xlabel('Timestamp', fontsize=fontsize)
axs[2].set_ylabel('Position [m]', fontsize=fontsize)
axs[2].legend(['z 1', 'z 2'])

#df_pose_1['roll'].plot(ax=axs[3])
#df_pose_2['roll'].plot(ax=axs[3])
#axs[3].set_xlabel('Timestamp', fontsize=fontsize)
#axs[3].set_ylabel('Angle [deg]', fontsize=fontsize)
#axs[3].legend(['roll 1', 'roll 2'])

#df_pose_1['pitch'].plot(ax=axs[4])
#df_pose_2['pitch'].plot(ax=axs[4])
#axs[4].set_xlabel('Timestamp', fontsize=fontsize)
#axs[4].set_ylabel('Angle [deg]', fontsize=fontsize)
#axs[4].legend(['pitch 1', 'pitch 2'])

#df_pose_1['yaw'].plot(ax=axs[5])
#df_pose_2['yaw'].plot(ax=axs[5])
#axs[5].set_xlabel('Timestamp', fontsize=fontsize)
#axs[5].set_ylabel('Angle [deg]', fontsize=fontsize)
#axs[5].legend(['yaw 1', 'yaw 2'])

NameError: name 'df_pose_1' is not defined

In [18]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib qt

fontsize=12
fig, axs = plt.subplots(nrows=2, sharex=True)

df_acc_bias_1[['x', 'y', 'z']].plot(ax=axs[0])
#df_acc_bias_2[['x', 'y', 'z']].plot(ax=axs[0], style='k--')
axs[0].set_xlabel('Timestamp', fontsize=fontsize)
axs[0].set_ylabel('Accelerometer bias [m/s**2]', fontsize=fontsize)
axs[0].legend(['x 1', 'y 1', 'z 1', 'x 2', 'y 2', 'z 2'])

df_gyro_bias_1[['x', 'y', 'z']].plot(ax=axs[1])
df_gyro_bias_2[['x', 'y', 'z']].plot(ax=axs[1], style='k--')
axs[1].set_xlabel('Timestamp', fontsize=fontsize)
axs[1].set_ylabel('Gyroscope bias [rad/s]', fontsize=fontsize)
axs[1].legend(['x 1', 'y 1', 'z 1', 'x 2', 'y 2', 'z 2'],  loc='upper right')